In [1]:
import os.path as op
import glob
import pandas as pd
import numpy as np
from mubench.utils.macro import CLASSIFICATION_DATASET, REGRESSION_DATASET
from mubench.utils.io import init_dir


In [12]:
CLASSIFICATION_METRICS = ['roc-auc', 'ece', 'nll', 'brier']
REGRESSION_METRICS = ['rmse', 'mae', 'nll', 'ce']
metrics_mapping = {
    'roc-auc': 'ROC-AUC',
    'ece': 'ECE',
    'nll': 'NLL',
    'brier': 'BS',
    'rmse': 'RMSE',
    'mae': 'MAE',
    'ce': 'CE'
}
MODEL_NAMES = [
    "DNN-rdkit",
    "ChemBERTa",
    "GROVER",
    "Uni-Mol"
]
model_mapping = {
    "DNN-rdkit": 'DNN',
    "ChemBERTa": "ChemBERTa",
    "GROVER": "GROVER",
    "Uni-Mol": "Uni-Mol"
}
CLASSIFICATION_UNCERTAINTY = [
    'none',
    'TemperatureScaling',
    'FocalLoss',
    'MCDropout',
    'SWAG',
    'BBP',
    'SGLD',
    'DeepEnsembles'
]
REGRESSION_UNCERTAINTY = [
    'none',
    'MCDropout',
    'SWAG',
    'BBP',
    'SGLD',
    'DeepEnsembles'
]
uncertainty_mapping = {
    'none': 'Deterministic',
    'TemperatureScaling': 'Temperature',
    'FocalLoss': 'Focal Loss',
    'MCDropout': 'MC Dropout',
    'SWAG': 'SWAG',
    'BBP': 'BBP',
    'SGLD': 'SGLD',
    'DeepEnsembles': 'Ensembles'
}
dataset_mapping = {
    "tox21": "Tox21",
    "esol": "ESOL",
    "freesolv": "FreeSolv",
    "lipo": "Lipophilicity",
    "muv": "MUV",
    "hiv": "HIV",
    "bace": "BACE",
    "bbbp": "BBBP",
    "toxcast": "ToxCast",
    "sider": "SIDER",
    "clintox": "ClinTox",
    "qm7": "QM7",
    "qm8": "QM8",
    "qm9": "QM9"
}

In [4]:
result_path = '../output/RESULTS/'
score_path = op.join(result_path, 'scores')
result_files = list(glob.glob(op.join(score_path, '*.csv')))


In [5]:
result_files

['../output/RESULTS/scores/ChemBERTa-muv.csv',
 '../output/RESULTS/scores/DNN-rdkit-lipo.csv',
 '../output/RESULTS/scores/DNN-rdkit-qm7.csv',
 '../output/RESULTS/scores/Uni-Mol-toxcast.csv',
 '../output/RESULTS/scores/Uni-Mol-muv.csv',
 '../output/RESULTS/scores/DNN-rdkit-bbbp.csv',
 '../output/RESULTS/scores/Uni-Mol-sider.csv',
 '../output/RESULTS/scores/GROVER-qm7.csv',
 '../output/RESULTS/scores/ChemBERTa-clintox.csv',
 '../output/RESULTS/scores/ChemBERTa-lipo.csv',
 '../output/RESULTS/scores/ChemBERTa-freesolv.csv',
 '../output/RESULTS/scores/ChemBERTa-bbbp.csv',
 '../output/RESULTS/scores/Uni-Mol-esol.csv',
 '../output/RESULTS/scores/Uni-Mol-qm7.csv',
 '../output/RESULTS/scores/GROVER-muv.csv',
 '../output/RESULTS/scores/ChemBERTa-tox21.csv',
 '../output/RESULTS/scores/GROVER-toxcast.csv',
 '../output/RESULTS/scores/Uni-Mol-freesolv.csv',
 '../output/RESULTS/scores/DNN-rdkit-tox21.csv',
 '../output/RESULTS/scores/ChemBERTa-qm7.csv',
 '../output/RESULTS/scores/GROVER-tox21.csv',
 '

## For classification datasets

In [16]:
mapped_metrics = [metrics_mapping[m] for m in CLASSIFICATION_METRICS]
for dataset in CLASSIFICATION_DATASET:
    dfs = list()
    for model_name in MODEL_NAMES:
        result_file = op.join(score_path, f"{model_name}-{dataset}.csv")
        df = pd.read_csv(result_file, index_col='method')
        for metrics in CLASSIFICATION_METRICS:
            df[metrics_mapping[metrics]] = [(f"{m:.4f}" if m == m else '(-)') + " " + (f"({s:.4f})" if s == s else '(-)')
                                            for m, s in zip(df[f'{metrics}-mean'], df[f'{metrics}-std'])]
        df = df[mapped_metrics]
        df = df.reindex([f'{model_name}-{unc}' for unc in CLASSIFICATION_UNCERTAINTY])
        index_mapping = {f"{model_name}-{unc}": uncertainty_mapping[unc] for unc in CLASSIFICATION_UNCERTAINTY}
        df = df.rename(index=index_mapping)
        index = pd.MultiIndex.from_tuples([(model_name, unc) for unc in df.index])
        df = df.set_index(index)
        dfs.append(df)

    data_df = pd.concat(dfs)
    data_df.to_latex(f"tb3.result.dataset.{dataset}.tex",
                     caption=f"Test results on {dataset_mapping[dataset]} in the format of ``metric mean (standard deviation)''.",
                     label=f"apptb:result.dataset.{dataset}",
                     position='tbh')



In [17]:
mapped_metrics = [metrics_mapping[m] for m in REGRESSION_METRICS]
for dataset in REGRESSION_DATASET:
    dfs = list()
    for model_name in MODEL_NAMES:
        result_file = op.join(score_path, f"{model_name}-{dataset}.csv")
        df = pd.read_csv(result_file, index_col='method')
        for metrics in REGRESSION_METRICS:
            df[metrics_mapping[metrics]] = [(f"{m:.4f}" if m == m else '(-)') + " " + (f"({s:.4f})" if s == s else '(-)')
                                            for m, s in zip(df[f'{metrics}-mean'], df[f'{metrics}-std'])]
        df = df[mapped_metrics]
        df = df.reindex([f'{model_name}-{unc}' for unc in REGRESSION_UNCERTAINTY])
        index_mapping = {f"{model_name}-{unc}": uncertainty_mapping[unc] for unc in REGRESSION_UNCERTAINTY}
        df = df.rename(index=index_mapping)
        index = pd.MultiIndex.from_tuples([(model_name, unc) for unc in df.index])
        df = df.set_index(index)
        dfs.append(df)

    data_df = pd.concat(dfs)
    data_df.to_latex(f"tb3.result.dataset.{dataset}.tex",
                     caption=f"Test results on {dataset_mapping[dataset]} in the format of ``metric mean (standard deviation)''.",
                     label=f"apptb:result.dataset.{dataset}",
                     position='tbh')

